## Data Handling

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import numpy as np
np.random.seed(4444)

In [2]:
df = pd.read_csv('../../../../../jaeyeun/01_nh_poc/15_split_data_set_and_make_json_for_train_test_set/split_70_15_15.csv')

df.head()

,file_name,label,raw_text,dataset_n,train_val_test
0,R1509261.txt,0,보 도 자 료\nhttp://www.msip.go.kr 보도일시 2015. 9. 4...,0,0
1,R2003733.txt,0,보도일시 2020. 3. 18.(수) 조간(온라인 3. 17. 12:00)부터 보도...,0,0
2,D1507076-1.txt,0,보 도 자 료\nhttp://www.msip.go.kr 보도일시 2015. 7. 1...,0,0
3,R2005031.txt,0,보 도 자 료\n배포일시 2020. 4. 29.(수) 총 4매(본문2) 담당 부서 ...,0,0
4,R2006226.txt,0,<전매체> 2020년 6월 3일(수) 10:00(국무회의 개최시)부터 보도하여 주시...,0,0


* Train : KDI 70+15 (train+val)
* Val : KDI 15(test)
* Test : legal 273(train+val+test)

In [3]:
X_train_df = df[(df['dataset_n'] == 0) & ((df['train_val_test'] == 0) | (df['train_val_test'] == 1))]
X_val_df = df[(df['dataset_n'] == 0) & (df['train_val_test'] == 2)]
X_test_df = df[df['dataset_n'] != 0]
y_train = df[(df['dataset_n'] == 0) & ((df['train_val_test'] == 0) | (df['train_val_test'] == 1))]['label']
y_val = df[(df['dataset_n'] == 0) & (df['train_val_test'] == 2)]['label']
y_test = df[df['dataset_n'] != 0]['label']

In [4]:
print(X_train_df.shape)
print(X_val_df.shape)
print(X_test_df.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(9174, 5)
(1626, 5)
(273, 5)
(9174,)
(1626,)
(273,)


In [5]:
n_features = 100000
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.9, # 0.9 만큼의 문서 이상에서 나오면 거른다.
    min_df=5, # 5개 미만의 문서에서 나오면 거른다.
    sublinear_tf = True, # tf value를 완만하게 처리 (outlier 처리 효과)
#     ngram_range = (1, 3),
    max_features=n_features)

In [6]:
X_train_tfidf = tfidf_vectorizer.fit(X_train_df['raw_text'])
# X_test_tfidf = tfidf_vectorizer.fit_transform(prep_text_test)
# X_test_hash = hash_vectorizer.fit_transform(prep_text_test)

In [7]:
X_train_tfidf_transformed = X_train_tfidf.transform(X_train_df['raw_text'])
X_val_tfidf_transformed = X_train_tfidf.transform(X_val_df['raw_text'])
X_test_tfidf_transformed = X_train_tfidf.transform(X_test_df['raw_text'])

In [8]:
print(X_train_tfidf_transformed.shape)
print(X_val_tfidf_transformed.shape)
print(X_test_tfidf_transformed.shape)

(9174, 96971)
(1626, 96971)
(273, 96971)


In [9]:
svd = TruncatedSVD(n_components=1000)
svd.fit(X_train_tfidf_transformed)

TruncatedSVD(n_components=1000)

In [10]:
X_train = svd.transform(X_train_tfidf_transformed)
X_val = svd.transform(X_val_tfidf_transformed)
X_test = svd.transform(X_test_tfidf_transformed)

In [11]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(9174, 1000)
(1626, 1000)
(273, 1000)


## Classify

* LightGBM RandomSearch

In [12]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgbm

In [13]:
lgbm_clf = lgbm.LGBMClassifier()

lgbm_param_grid = {'learning_rate': [.01, .015, .025, .05, .1],
#                   'Gamma': [.05, .1, .3, .5, .7, .9, 1],
                  'max_depth': [3, 5, 7, 9, 12, 15, 17, 25],
                  'min_child_weight': [1, 3, 5, 7],
                  'subsample': np.linspace(0.6, 1, 5)}

fit_params = {"early_stopping_rounds" : 20,
             "eval_metric" : "multi_error",
             "eval_set" : [(X_train, y_train), (X_val, y_val)]}

# Create a random search object
lgbm_random = RandomizedSearchCV(estimator = lgbm_clf,
                                param_distributions = lgbm_param_grid,
                                n_iter = 20, # n_iters in param combinations
                                scoring='accuracy',
                                n_jobs=-1,
                                cv = 5,
                                refit=True,
                                return_train_score = True,
                                verbose=10)

# Fit to the training data
lgbm_random.fit(X_train, y_train, **fit_params)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 12.5min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-1)]: Done  48 out of 100 | elapsed: 22.1min remaining: 24.0min
[Parallel(n_jobs=-1)]: Done  59 out of 100 | elapsed: 25.9min remaining: 18.0min
[Parallel(n_jobs=-1)]: Done  70 out of 100 | elapsed: 29.8min remaining: 12.8min
[Parallel(n_jobs=-1)]: Done  81 out of 100 | elapsed: 34.7min remaining:  8.1min
[Parallel(n_jobs=-1)]: Done  92 out of 100 | elapsed: 37.3min remaining:  3.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 39.2min finished


[1]	training's multi_error: 0.490735	training's multi_logloss: 1.99096	valid_1's multi_error: 0.49877	valid_1's multi_logloss: 2.01396
Training until validation scores don't improve for 20 rounds
[2]	training's multi_error: 0.313168	training's multi_logloss: 1.80062	valid_1's multi_error: 0.367159	valid_1's multi_logloss: 1.84469
[3]	training's multi_error: 0.245258	training's multi_logloss: 1.64734	valid_1's multi_error: 0.311193	valid_1's multi_logloss: 1.71122
[4]	training's multi_error: 0.214846	training's multi_logloss: 1.52048	valid_1's multi_error: 0.279213	valid_1's multi_logloss: 1.6014
[5]	training's multi_error: 0.194245	training's multi_logloss: 1.41169	valid_1's multi_error: 0.266298	valid_1's multi_logloss: 1.50947
[6]	training's multi_error: 0.180183	training's multi_logloss: 1.3162	valid_1's multi_error: 0.253998	valid_1's multi_logloss: 1.42649
[7]	training's multi_error: 0.169719	training's multi_logloss: 1.23203	valid_1's multi_error: 0.242927	valid_1's multi_logloss

RandomizedSearchCV(cv=5, estimator=LGBMClassifier(), n_iter=20, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.015, 0.025,
                                                          0.05, 0.1],
                                        'max_depth': [3, 5, 7, 9, 12, 15, 17,
                                                      25],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'subsample': array([0.6, 0.7, 0.8, 0.9, 1. ])},
                   return_train_score=True, scoring='accuracy', verbose=10)

In [14]:
pred_train = lgbm_random.best_estimator_.predict(X_train)
pred_val = lgbm_random.best_estimator_.predict(X_val)
pred_test = lgbm_random.best_estimator_.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))
print(accuracy_score(y_test, pred_test))

0.9991279703509919
0.8191881918819188
0.16483516483516483


In [16]:
lgbm_random.best_estimator_

LGBMClassifier(max_depth=7, min_child_weight=7)

In [17]:
cv_result_df = pd.DataFrame(lgbm_random.cv_results_)

df_list = []
for i in range(20):
    df_list.append(pd.DataFrame([cv_result_df.loc[i, "params"]]))
    
param_table = pd.concat(df_list)

param_table['mean_test_score'] = cv_result_df['mean_test_score'].values

param_table.sort_values(by='mean_test_score', axis=0)

,subsample,min_child_weight,max_depth,learning_rate,mean_test_score
0,1.0,3,5,0.015,0.762481
0,0.9,1,7,0.010,0.764007
0,0.7,7,15,0.015,0.777523
0,0.9,7,5,0.025,0.778395
0,0.8,3,15,0.015,0.779704
0,1.0,3,17,0.015,0.780358
0,1.0,5,12,0.015,0.780685
0,0.6,3,12,0.025,0.791476
0,0.6,7,5,0.050,0.798561
0,1.0,3,7,0.050,0.802268
